In [68]:
import pandas as pd
import streamlit_authenticator as stauth

In [69]:
data = pd.read_excel('pegawai110.xlsx', dtype={'NIP':'str','KD':'str'},usecols=['NIP','Nama','Kantor','KD'])

In [70]:
data['password'] = 'KilauJaktim'+'@'+data['NIP']

In [71]:
data

,Nama,NIP,Kantor,KD,password
0,MUHAMMAD ISMIRANSYAH M. ZAIN,060077930,Kanwil DJP Jakarta Timur,110,KilauJaktim@060077930
1,SUGENG SATOTO,060094070,Kanwil DJP Jakarta Timur,110,KilauJaktim@060094070
2,DESSY EKA PUTRI,060087065,Kanwil DJP Jakarta Timur,110,KilauJaktim@060087065
3,SANDRA BUANA,060083161,Kanwil DJP Jakarta Timur,110,KilauJaktim@060083161
4,ARDHIE PERMADI,060083105,Kanwil DJP Jakarta Timur,110,KilauJaktim@060083105
...,...,...,...,...,...
1082,NASTRIA PRATAMA,930302289,KPP Madya Dua Jakarta Timur,097,KilauJaktim@930302289
1083,ADRY NOVIANTO,860013856,KPP Madya Dua Jakarta Timur,097,KilauJaktim@860013856
1084,HERU IRFANTO,808320132,KPP Madya Dua Jakarta Timur,097,KilauJaktim@808320132
1085,MIFTAKHUL PRATAMA PUTRA,810600567,KPP Madya Dua Jakarta Timur,097,KilauJaktim@810600567


In [72]:
def makehash(x):
    passw = stauth.Hasher(x).generate()
    return passw

In [73]:
data = data.head()

In [74]:
data['pass_hashed'] = data['password'].apply(lambda x: makehash([x])[0])

C:\Users\sugengw07\AppData\Local\Temp\ipykernel_12588\584411665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['pass_hashed'] = data['password'].apply(lambda x: makehash([x])[0])


In [17]:
secrets = pd.DataFrame(pass_hashed, index=data.NIP.tolist(),columns=['PASS_HASHED'])

In [20]:
credential = pd.merge(data, secrets, left_on='NIP', right_index=True, how='inner')

In [76]:
credential =data

In [78]:
import duckdb
db = duckdb.connect('credentials.duckdb')

In [79]:
db.execute('drop table credential ')

CatalogException: Catalog Error: Table with name credential does not exist!
Did you mean "information_schema.schemata"?

In [64]:
db.execute('create table credentials from credential')

ParserException: Parser Error: syntax error at or near "from"
LINE 1: create table credentials from credential
                                 ^

In [97]:
db.close()

In [84]:
s=db.execute('select * from credential').df()


In [96]:
for passq in s['pass_hashed']:
    print(passq)

$2b$12$NHHMlYdHHHk3gjEqcJ71medYmF4PN/zdtJMXqlp.G25YG2xeeuoq2
$2b$12$uHZzjj0lUx8m.pdyiQwzoe7BpC8YeEiNqTg7IDJRj86FybpVOoF62
$2b$12$ReC9emPw5nKWoiQ4ePrEKeZpv88MTldXKk.fybMsR1nGb0fWNk1Gy
$2b$12$AGBBHqFhLXFKGzbN.Simou2WXWqfRqH8Opwcc.PXyu0jtcnXrSFem
$2b$12$bOg757ifo8.NLExn1rh9a.IXnvVZ863Zb73TphnYXMjeHBzjLFMaO
